# TRIAL NOTEBOOK

## Import Libraries

In [1]:
import os
import time
import pandas as pd
import nltk

## Download NLTK Resources

In [2]:
# !mkdir -p ~/nltk_data
# !git clone https://github.com/nltk/nltk_data.git
# !mv nltk_data/packages/* ~/nltk_data/
# !rm -rf nltk_data

# Unzip punkt
# !WORK_DIR=$(pwd)
# !cd ~/nltk_data/tokenizers/
# !unzip punkt.zip
# !rm punkt.zip
# !cd $WORK_DIR

## Import helper functions

In [3]:
from utils import load_sessions, read_session, read_file
from operations import build_text

## Read File and Load Events

In [4]:
events = read_file("7c1e066d0c9c4901921cab117a0b7e73.jsonl")

Successfully read 1504 events in a writing session from ./coauthor-v1.0/7c1e066d0c9c4901921cab117a0b7e73.jsonl


## Build Text Buffer

In [5]:
text_buffer = []

In [6]:
for event in events:
    buffer = build_text(text_buffer, event)
    text_buffer.append(buffer)

## Initialize DataFrame

In [7]:
df = pd.DataFrame()

In [8]:
df["text_buffer"] = text_buffer
df["events"] = events

In [9]:
df.head()

,text_buffer,events
0,"A woman has been dating guy after guy, but it ...","{'eventName': 'system-initialize', 'eventSourc..."
1,"A woman has been dating guy after guy, but it ...","{'eventName': 'text-insert', 'eventSource': 'u..."
2,"A woman has been dating guy after guy, but it ...","{'eventName': 'text-insert', 'eventSource': 'u..."
3,"A woman has been dating guy after guy, but it ...","{'eventName': 'suggestion-get', 'eventSource':..."
4,"A woman has been dating guy after guy, but it ...","{'eventName': 'suggestion-open', 'eventSource'..."


## Extract Sentences

In [11]:
# Add sentences

sentence_buffer = []

for text in df["text_buffer"]:
    sentences = nltk.tokenize.sent_tokenize(text)
    sentence_buffer.append(sentences)